# Indexing




In [28]:
# import tiktoken to count the number of tokens.
import tiktoken
import os 
from dotenv import load_dotenv,find_dotenv

OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
url="https://aiproxy.sanand.workers.dev/openai/v1"



In [29]:
# document 
question="What is my favourite fruit?"
doc="My favourite fruit is mango."

In [31]:
# function to count tokens
def count_tokens(string: str, encoding_name: str) -> int:
    encoding = tiktoken.get_encoding(encoding_name)
    tokens = encoding.encode(string)
    return len(tokens)

count_tokens(question, "cl100k_base")



6

In [ ]:
from langchain_openai import OpenAIEmbeddings

embed = OpenAIEmbeddings(
    openai_api_key= OPENAI_API_KEY,
    openai_api_base=url,
    model="text-embedding-3-small"

)

query_result = embed.embed_query(question)
document_result = embed.embed_query(doc)
print(len(document_result))

In [36]:
# find the cosine similarity
import numpy as np
def cosine_sim(vec1,vec2):
    dot_prod=np.dot(vec1,vec2)
    v1_norm=np.linalg.norm(vec1)
    v2_norm=np.linalg.norm(vec2)
    return dot_prod /(v1_norm*v2_norm)

similarity=cosine_sim(query_result,document_result)
print(f"cosine similarity= {similarity}")

cosine similarity= 0.7284394453560288


In [39]:
#### INDEXING ####

# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

In [40]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

In [42]:
# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, embedding=embed)

retriever = vectorstore.as_retriever()

# Retrieval

In [43]:
# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, embedding=embed)


retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
docs = retriever.get_relevant_documents("What is Task Decomposition?")
len(docs)

/var/folders/qc/8f8k87s13ld9rf4lclv6h8n80000gn/T/ipykernel_21213/1105722730.py:8: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents("What is Task Decomposition?")


1

# Generation


In [45]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Custom LLM setup with base and token
llm = ChatOpenAI(
    model_name="gpt-4o-mini",  # or the correct model your server supports
    temperature=0,
    openai_api_key=OPENAI_API_KEY,
    openai_api_base=url,
)

# Chain setup
chain: Runnable = prompt | llm

# Sample run
response = chain.invoke({
    "context": docs,
    "question": "What is Task Decomposition?"
})
print(response.content)

Task Decomposition is the process of breaking down a complicated task into smaller, more manageable steps. It involves techniques like Chain of Thought (CoT), which encourages a model to think step by step to enhance performance on complex tasks. CoT transforms large tasks into simpler ones, providing insight into the model's reasoning process. Additionally, the Tree of Thoughts approach extends CoT by exploring multiple reasoning possibilities at each step, creating a tree structure of thoughts. Task decomposition can be achieved through simple prompting, task-specific instructions, or human inputs.


In [46]:
from langchain import hub
prompt_hub_rag = hub.pull("rlm/rag-prompt")
prompt_hub_rag

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

/Users/sarvagra/Library/Python/3.9/lib/python/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


'Task Decomposition is the process of breaking down a complicated task into smaller, more manageable steps. It involves techniques like Chain of Thought (CoT), which encourages a model to think step by step to enhance performance on complex tasks, and Tree of Thoughts, which explores multiple reasoning possibilities at each step by creating a tree structure of thoughts. Task decomposition can be achieved through simple prompting, task-specific instructions, or human inputs.'